In [ ]:
import phd

import os
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import seaborn as sns
import nengo
import numpy as np
import soundfile as sf
from IPython.display import Audio

# Some plotting niceties
phd.plots.setup()

def run_and_plot(model, name, t):
    net = model.build()
    with net:
        p_syll = nengo.Probe(net.sequence.syllable.output, synapse=0.01)
        p_syllnext = nengo.Probe(net.sequence.syllable_next.output, synapse=0.01)
        p_gate = nengo.Probe(net.sequencer.gate, synapse=0.01)
        p_reset = nengo.Probe(net.sequencer.reset, synapse=0.01)
        p_timer = nengo.Probe(net.sequencer.timer, synapse=0.01)
        p_disinhib = [nengo.Probe(dmp.disinhibit, synapse=0.01) for dmp in net.syllables]
        p_dmps = [nengo.Probe(dmp.osc, synapse=0.01) for dmp in net.syllables]
        p_prod = nengo.Probe(net.production_info.output, synapse=0.01)
    sim = nengo.Simulator(net)
    sim.run(t)

    # A bit of analysis
    syllables = [s[1].upper() for s in freqs]
    reconstructed = phd.experiments.gesture_score(sim.data[p_prod], model.trial.dt)
    audio, fs = reconstructed.synthesize()
    rtraj = reconstructed.trajectory(model.trial.dt)
 
    accuracy, n_sub, n_del, n_ins = phd.analysis.gs_accuracy(reconstructed, gs_targets)
    timing_mean, timing_var = phd.analysis.gs_timing(reconstructed, gs_targets)
    cooccur, co_chance = phd.analysis.gs_cooccur(reconstructed, gs_targets)

    # Print results
    print("accuracy: %f" % accuracy)
    print("%d substitutions, %d deletions, %d insertions" % (n_sub, n_ins, n_del))
    print("timing: %f +/- %f" % (float(timing_mean), float(timing_var)))
    print("cooccurring: %f, chance is %f" % (float(cooccur), float(co_chance)))

    fig = plt.figure(figsize=(12, 12))
    grid = gs.GridSpec(12, 3, width_ratios=[1, 1, 1.4])

    # Current syllable output
    ax = plt.subplot(grid[:3, 0])
    ax.plot(sim.trange(), 
            nengo.spa.similarity(sim.data[p_syll],
                                 net.sequence.vocab.create_subset(syllables)))
    ax.legend(syllables, loc='best')
    ax.set_title("Current syllable")
    ax.set_ylabel("Similarity")
    ax.set_xticks(())
    sns.despine(ax=ax)

    # Next syllable output
    ax = plt.subplot(grid[3:6, 0])
    ax.plot(sim.trange(), 
            nengo.spa.similarity(sim.data[p_syllnext],
                                 net.sequence.vocab.create_subset(syllables)))
    ax.legend(syllables, loc='best')
    ax.set_title("Next syllable")
    ax.set_ylabel("Similarity")
    ax.set_xticks(())
    sns.despine(ax=ax)

    # Disinhibition
    ax = plt.subplot(grid[6:9, 0])
    for p_dis in p_disinhib:
        ax.plot(sim.trange(), sim.data[p_dis])
    ax.legend(syllables, loc="best")
    ax.set_title("DMP disinhibition")
    ax.set_ylabel("Disinhibition strength")
    ax.set_xticks(())
    sns.despine(ax=ax)

    # Gate / reset / timer dims
    ax = plt.subplot(grid[9:, 0])
    ax.plot(sim.trange(), sim.data[p_gate])
    ax.plot(sim.trange(), sim.data[p_reset])
    ax.plot(sim.trange(), sim.data[p_timer])
    ax.legend(['Gate', 'Reset', 'Timer', 'Timer'],
              loc="lower left", frameon=True)
    ax.set_title("Sequencer control signals")
    ax.set_ylabel("Decoded output")
    ax.set_xlabel("Time (s)")
    sns.despine(ax=ax)

    # Oscillator states
    ax = plt.subplot(grid[:3, 1])
    ax.plot(sim.data[p_timer].T[0], sim.data[p_timer].T[1])
    ax.set_title("Timer state")
    ax.set_ylabel("$x_1$")
    ax.set_xlabel("$x_0$")
    sns.despine(ax=ax)
    for i, (label, p_dmp) in enumerate(zip(syllables, p_dmps)):
        ax = plt.subplot(grid[(i+1)*3:(i+2)*3, 1])
        ax.plot(sim.data[p_dmp].T[0], sim.data[p_dmp].T[1])
        ax.set_title("%s DMP state" % label)
        ax.set_ylabel("$x_1$")
        ax.set_xlabel("$x_0$")
        sns.despine(ax=ax)

    # Gesture trajectories
    ax = plt.subplot(grid[:4, 2])
    phd.plots.plot_traj(phd.experiments.ideal_traj(model, syllables), ax=ax, cbar=False)
    ax.set_title("Original gesture trajectory (target)")

    ax = plt.subplot(grid[4:8, 2])
    phd.plots.plot_traj(sim.data[p_prod], ax=ax, cbar=False)
    ax.set_title("Simulated gesture trajectory")

    ax = plt.subplot(grid[8:, 2])
    phd.plots.plot_traj(rtraj, ax=ax, cbar=False)
    ax.set_title("Reconstructed gesture trajectory")

    fig.tight_layout()
    phd.plots.savefig(fig, 'results', name)
    return accuracy, audio

## Basic usage

In [ ]:
model = phd.sermo.Production()
model.trial.sequence = 'BLAA*POS1 + TII*POS2 + DAS*POS3'
freqs = [('ges-de-ccv', 'blaa', 1.1), ('ges-de-cv', 'tii', 2.1), ('ges-de-cvc', 'das', 1.5)]
t = 0.3
audio = []
gs_targets = []
for gdir, ges, freq in freqs:
    path = phd.ges_path(gdir, '%s.ges' % ges.lower())
    score = phd.vtl.parse_ges(path)
    a, fs = score.synthesize()
    audio.append(a)
    gs_targets.append(score)
    traj = score.trajectory(dt=model.trial.dt)
    model.add_syllable(label=ges.upper(), freq=freq, trajectory=traj)
    t += 1. / freq
audio = np.vstack(audio)

In [ ]:
sf.write(audio, 'original.wav', fs)
Audio(data=audio.ravel(), rate=fs)

In [ ]:
acc = 0.
while acc < 0.9:
    acc, audio = run_and_plot(model, 'prod-good', t)
sf.write(audio, 'synthesized.wav', fs)
Audio(data=audio.ravel(), rate=fs)

In [ ]:
sf.write(audio, 'synthesized.wav', fs)
Audio(data=audio.ravel(), rate=fs)

In [ ]:
acc = 1.
while acc > 0.8:
    acc, audio = run_and_plot(model, 'prod-bad', t)
Audio(data=audio.ravel(), rate=fs)

## VTL gestures

In [ ]:
labels = phd.vtl.VTL().gesture_labels()
labels.remove('f0')
for i, l in enumerate(labels):
    print("%s, %s" % (l, phd.experiments.ix2seqlabel(i, labels)))

## Running an experiment

In [ ]:
model = phd.sermo.Production()
model.syllable.tau = 0.018
expt = phd.experiments.ProductionExperiment(model, n_syllables=2, sequence_len=2)
key = expt.run()
res = phd.experiments.ProductionResult.load(key)

In [ ]:
# Actual trajectory (from .ges files)
img(res.traj)
print(res.seq)
print(res.freqs)

In [ ]:
# Simulated trajectory
img(res.simtraj)
print(res.simrmse)

In [ ]:
# Trajectory from .ges file reconstructed from simulated trajectory
img(res.reconstructed)
print(res.reconstructedrmse)

In [ ]:
Audio(data=res.clean_audio.flatten(), rate=res.fs)

In [ ]:
# Audio synthesized by VTL given .ges file
# reconstructed from simulated trajectory;
# note, no f0 gestures provided.
Audio(data=res.audio.flatten(), rate=res.fs)

In [ ]:
print("accuracy: %f" % res.accuracy)
print("timing: %f +/- %f" % (float(res.timing_mean), float(res.timing_var)))
print("cooccurring: %f, chance is %f" % (float(res.cooccur), float(res.co_chance)))

## Plotting experimental results

In [ ]:
phd.plots.prod_time('tau', 'Syllable tau');

In [ ]:
phd.plots.prod_time('syllneurons', 'Syllable neurons');

In [ ]:
phd.plots.prod_time('seqneurons', 'Sequencer neurons');

In [ ]:
phd.plots.prod_time('freq', 'Frequency');

In [ ]:
phd.plots.prod_time('n_syllables', 'Syllabary size');

In [ ]:
phd.plots.prod_time('sequence_len', 'Sequence length');

In [ ]:
phd.plots.prod_cmp('repeat', 'Repeated syllables', hue_order=["False", "True"]);

In [ ]:
df = phd.analysis.load_results(
    phd.experiments.ProductionResult,
    keys=['seqneurons'])

for i in range(len(df.index)):
    if df['seqneurons'][i] != '1000':
        continue

    # Output all the .wavs to current directory
    cpath, spath = '%d-clean.wav' % i, '%d-sim.wav' % i
    if not os.path.exists(cpath):
        sf.write(df['clean_audio'][i], cpath, df['fs'][i])
    if not os.path.exists(spath):
        sf.write(df['audio'][i], spath, df['fs'][i])

# After going through them manually...
good = [22, 23, 24, 25, 26, 35, 38]
okay = [20, 21, 27, 28, 29, 31, 33, 34]
bad = [30, 32, 36, 37, 39]
total = len(good) + len(okay) + len(bad)
print("Good: %d, %f" % (len(good), float(len(good)) / total))
print("Okay: %d, %f" % (len(okay), float(len(okay)) / total))
print("Bad: %d, %f" % (len(bad), float(len(bad)) / total))

## Scaling

In [ ]:
def n_neurons(msg, model, n_syllables):
    net = model.build()
    nn = sum(e.n_neurons for e in net.all_ensembles)
    print("=== %s ===" % msg)

    # --- SPA sequence stuff
    spasyllneurons = model.sequence.syllable_d * model.sequence.n_per_d
    spaneurons = spasyllneurons * 5
    spaneurons += len(net.sequence.bind.all_ensembles) * model.sequence.n_per_d
    # * 5 = sequence, pos (2), pos_next (2), 
    # net.sequence.bind.all_ensembles => .5 * n_per_d per ensemble, but there's both bind and bind_next
    amneurons = 2 * ((n_syllables * 3 + 1) * 50)  # 50 is default for AM
    # 2: syllable and syllable_next
    # + 1 is for the default output
    spaneurons += amneurons

    # --- Sequencer stuff
    # * 2 + 2: timer is 2d; timer_recur is 2d
    # + 60: reset is 60 neurons
    # + 20: tr_inhibit is 20 neurons
    # + 60: gate is 60 neurons
    seqneurons = (model.sequencer.n_per_d * (2 + 2)) + 60 + 20 + 60

    # --- DMP stuff
    syllneurons = model.syllable.n_per_d
    # * (2 + 1): osc is 2d; diff is 1
    # + 20: diff_inhib is 20
    # + 20: inhib is 20
    dmpneurons = n_syllables * ((syllneurons * (2 + 1)) + 20 + 20)

    # Emulate having more than 3 syllables

    # --- Readout
    prodneurons = model.production_info.n_per_d * 48

    # --- Total
    allneurons = spaneurons + seqneurons + dmpneurons + prodneurons

    print("Nspa: %d, Nseq: %d, Ndmp: %d, Nprod: %d" % (
        spaneurons, seqneurons, dmpneurons, prodneurons))
    print("Total: %d neurons" % allneurons)
    print("%.3f mm^3 of cortex" % (allneurons / 27000.))
    print("")

def add_syllables(model, n_syllables):
    model.trial.sequence = "POS1"
    for p, f in zip(*phd.analysis.get_syllables(n_syllables, 1, 1)):
        tr = phd.vtl.parse_ges(p).trajectory(model.trial.dt)
        lbl = phd.experiments.path2label(p)
        model.add_syllable(label=lbl, freq=f, trajectory=tr)

model = phd.sermo.Production()
add_syllables(model, 3)
net = model.build()
n_neurons("Default configuration", model, 3)
model = phd.sermo.Production()
model.sequence.syllable_d = 256
add_syllables(model, 3)
net = model.build()
n_neurons("Conservative estimate", model, 1000)
model = phd.sermo.Production()
model.sequence.syllable_d = 512
add_syllables(model, 3)
net = model.build()
n_neurons("Generous estimate", model, 2000);